In [2]:
#Import dependencies
import pandas as pd
import sqlite3
import argparse
import json
import pprint
import requests
import sys
import urllib
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode
import matplotlib.pyplot as plt
import csv
import os

In [13]:

crime_df_21 = pd.read_csv('Crime_Incidents_in_2021 (1).csv')
crime_df_20 = pd.read_csv('Crime_Incidents_in_2020 (1).csv')

frames = [crime_df_21,crime_df_20]
crime_df = pd.concat(frames)
#crime_df = crime_df.dropna()
crime_df.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.016427,38.945357,20139232,2021/12/06 05:00:00+00,MIDNIGHT,GUN,HOMICIDE,300 - 399 BLOCK OF ALLISON STREET NW,398576.16,141939.53,...,002301 2,2301.0,Precinct 46,38.945349,-77.016425,NaN,2020/09/29 19:59:32+00,NaN,217939630,NaN
1,-77.068637,38.932083,11048862,2021/10/13 21:24:42+00,EVENING,OTHERS,THEFT F/AUTO,3100 - 3199 BLOCK OF 35TH STREET NW,394049.03,140468.17,...,000600 3,600.0,Precinct 27,38.932075,-77.068635,NaN,2021/10/13 14:00:00+00,2021/10/13 15:15:00+00,217945488,NaN
2,-76.926220,38.884377,10008883,2021/08/04 20:36:50+00,EVENING,OTHERS,THEFT/OTHER,300 - 348 BLOCK OF 53RD STREET SE,406401.56,135172.74,...,009905 1,9905.0,Precinct 105,38.884369,-76.926218,NaN,2021/07/16 16:00:00+00,2021/07/20 16:05:00+00,217949732,NaN
3,-76.989343,38.905932,11124202,2021/09/01 12:39:59+00,DAY,OTHERS,THEFT/OTHER,1100 - 1199 BLOCK OF OATES STREET NE,400924.55,137563.02,...,008802 3,8802.0,Precinct 77,38.905924,-76.989341,NaN,2021/09/01 01:36:00+00,NaN,217949735,NaN
4,-76.985593,38.912527,12113231,2021/08/12 12:00:17+00,DAY,OTHERS,THEFT/OTHER,1800 - 1899 BLOCK OF CENTRAL PLACE NE,401249.74,138295.18,...,008803 1,8803.0,Precinct 76,38.912520,-76.985590,NaN,2021/08/11 14:30:00+00,2021/08/12 04:00:00+00,217949736,NaN


In [14]:
crime_df.columns


Index(['X', 'Y', 'CCN', 'REPORT_DAT', 'SHIFT', 'METHOD', 'OFFENSE', 'BLOCK',
       'XBLOCK', 'YBLOCK', 'WARD', 'ANC', 'DISTRICT', 'PSA',
       'NEIGHBORHOOD_CLUSTER', 'BLOCK_GROUP', 'CENSUS_TRACT',
       'VOTING_PRECINCT', 'LATITUDE', 'LONGITUDE', 'BID', 'START_DATE',
       'END_DATE', 'OBJECTID', 'OCTO_RECORD_ID'],
      dtype='object')

In [15]:
# Drop unnecessary crime columns for machine learning
crime_df = crime_df.drop(['X','Y', 
                          'CCN','METHOD',
                          'XBLOCK','YBLOCK','ANC',
                          'BLOCK_GROUP','CENSUS_TRACT',
                          'BID','START_DATE','END_DATE',
                          'OBJECTID','OCTO_RECORD_ID'
                         ], axis=1)
crime_df.head()

,REPORT_DAT,SHIFT,OFFENSE,BLOCK,WARD,DISTRICT,PSA,NEIGHBORHOOD_CLUSTER,VOTING_PRECINCT,LATITUDE,LONGITUDE
0,2021/12/06 05:00:00+00,MIDNIGHT,HOMICIDE,300 - 399 BLOCK OF ALLISON STREET NW,4,4.0,407.0,Cluster 18,Precinct 46,38.945349,-77.016425
1,2021/10/13 21:24:42+00,EVENING,THEFT F/AUTO,3100 - 3199 BLOCK OF 35TH STREET NW,3,2.0,204.0,Cluster 15,Precinct 27,38.932075,-77.068635
2,2021/08/04 20:36:50+00,EVENING,THEFT/OTHER,300 - 348 BLOCK OF 53RD STREET SE,7,6.0,604.0,Cluster 33,Precinct 105,38.884369,-76.926218
3,2021/09/01 12:39:59+00,DAY,THEFT/OTHER,1100 - 1199 BLOCK OF OATES STREET NE,5,5.0,506.0,Cluster 23,Precinct 77,38.905924,-76.989341
4,2021/08/12 12:00:17+00,DAY,THEFT/OTHER,1800 - 1899 BLOCK OF CENTRAL PLACE NE,5,5.0,506.0,Cluster 23,Precinct 76,38.912520,-76.985590


In [16]:
crime_df.count()

REPORT_DAT              56233
SHIFT                   56233
OFFENSE                 56233
BLOCK                   56233
WARD                    56233
DISTRICT                56029
PSA                     56013
NEIGHBORHOOD_CLUSTER    56233
VOTING_PRECINCT         56233
LATITUDE                56233
LONGITUDE               56233
dtype: int64

In [17]:
crime_df = crime_df.dropna()
crime_df

,REPORT_DAT,SHIFT,OFFENSE,BLOCK,WARD,DISTRICT,PSA,NEIGHBORHOOD_CLUSTER,VOTING_PRECINCT,LATITUDE,LONGITUDE
0,2021/12/06 05:00:00+00,MIDNIGHT,HOMICIDE,300 - 399 BLOCK OF ALLISON STREET NW,4,4.0,407.0,Cluster 18,Precinct 46,38.945349,-77.016425
1,2021/10/13 21:24:42+00,EVENING,THEFT F/AUTO,3100 - 3199 BLOCK OF 35TH STREET NW,3,2.0,204.0,Cluster 15,Precinct 27,38.932075,-77.068635
2,2021/08/04 20:36:50+00,EVENING,THEFT/OTHER,300 - 348 BLOCK OF 53RD STREET SE,7,6.0,604.0,Cluster 33,Precinct 105,38.884369,-76.926218
3,2021/09/01 12:39:59+00,DAY,THEFT/OTHER,1100 - 1199 BLOCK OF OATES STREET NE,5,5.0,506.0,Cluster 23,Precinct 77,38.905924,-76.989341
4,2021/08/12 12:00:17+00,DAY,THEFT/OTHER,1800 - 1899 BLOCK OF CENTRAL PLACE NE,5,5.0,506.0,Cluster 23,Precinct 76,38.912520,-76.985590
...,...,...,...,...,...,...,...,...,...,...,...
27907,2020/02/17 18:03:27+00,DAY,THEFT F/AUTO,2200 - 2224 BLOCK OF MARTIN LUTHER KING JR AVE...,8,7.0,701.0,Cluster 28,Precinct 114,38.864380,-76.990009
27908,2020/02/17 18:15:15+00,DAY,THEFT F/AUTO,2600 - 2699 BLOCK OF ADAMS MILL ROAD NW,1,3.0,303.0,Cluster 1,Precinct 35,38.923676,-77.044240
27909,2020/02/17 19:11:24+00,DAY,THEFT F/AUTO,600 - 699 BLOCK OF I STREET NW,2,1.0,101.0,Cluster 8,Precinct 129,38.900868,-77.020910
27910,2020/02/17 20:18:06+00,EVENING,THEFT F/AUTO,2201 - 2299 BLOCK OF 10TH STREET NW,1,3.0,305.0,Cluster 3,Precinct 22,38.919857,-77.025993


In [18]:
crime_df.count()

REPORT_DAT              56012
SHIFT                   56012
OFFENSE                 56012
BLOCK                   56012
WARD                    56012
DISTRICT                56012
PSA                     56012
NEIGHBORHOOD_CLUSTER    56012
VOTING_PRECINCT         56012
LATITUDE                56012
LONGITUDE               56012
dtype: int64

In [19]:
# Most crimes are commited in which District --- answer 3, 2, 5 are the top 3 districts with the highest crime with district 7 the least amount of crime
districts = crime_df.DISTRICT.value_counts()
districts.head(10)

3.0    10539
2.0     9706
5.0     8837
1.0     7940
6.0     7350
4.0     7132
7.0     4508
Name: DISTRICT, dtype: int64

In [20]:
#Police Service Areas with top crime rate
psa = crime_df.PSA.value_counts()
psa.head(10)

305.0    1979
208.0    1967
501.0    1742
302.0    1739
106.0    1725
307.0    1683
207.0    1652
206.0    1587
104.0    1523
506.0    1376
Name: PSA, dtype: int64

In [21]:
#Police Shift with the most crime -- EVENING is when the most crimes are committed
shift = crime_df.SHIFT.value_counts()
shift.head(10)

EVENING     24346
DAY         21062
MIDNIGHT    10604
Name: SHIFT, dtype: int64

In [22]:
#count of offenses 2020 and 2021 -- Theft appears to be the most popular crime
crime_offense = crime_df.OFFENSE.value_counts() 
crime_offense.head(10)

THEFT/OTHER                   21733
THEFT F/AUTO                  16874
MOTOR VEHICLE THEFT            6702
ROBBERY                        4012
ASSAULT W/DANGEROUS WEAPON     3285
BURGLARY                       2611
HOMICIDE                        420
SEX ABUSE                       358
ARSON                            17
Name: OFFENSE, dtype: int64

In [23]:
# Use Pandas get_dummies to convert categorical data
data_binary_encoded = pd.get_dummies(crime_df, columns=["OFFENSE","SHIFT"])
data_binary_encoded.head()

,REPORT_DAT,BLOCK,WARD,DISTRICT,PSA,NEIGHBORHOOD_CLUSTER,VOTING_PRECINCT,LATITUDE,LONGITUDE,OFFENSE_ARSON,...,OFFENSE_BURGLARY,OFFENSE_HOMICIDE,OFFENSE_MOTOR VEHICLE THEFT,OFFENSE_ROBBERY,OFFENSE_SEX ABUSE,OFFENSE_THEFT F/AUTO,OFFENSE_THEFT/OTHER,SHIFT_DAY,SHIFT_EVENING,SHIFT_MIDNIGHT
0,2021/12/06 05:00:00+00,300 - 399 BLOCK OF ALLISON STREET NW,4,4.0,407.0,Cluster 18,Precinct 46,38.945349,-77.016425,0,...,0,1,0,0,0,0,0,0,0,1
1,2021/10/13 21:24:42+00,3100 - 3199 BLOCK OF 35TH STREET NW,3,2.0,204.0,Cluster 15,Precinct 27,38.932075,-77.068635,0,...,0,0,0,0,0,1,0,0,1,0
2,2021/08/04 20:36:50+00,300 - 348 BLOCK OF 53RD STREET SE,7,6.0,604.0,Cluster 33,Precinct 105,38.884369,-76.926218,0,...,0,0,0,0,0,0,1,0,1,0
3,2021/09/01 12:39:59+00,1100 - 1199 BLOCK OF OATES STREET NE,5,5.0,506.0,Cluster 23,Precinct 77,38.905924,-76.989341,0,...,0,0,0,0,0,0,1,1,0,0
4,2021/08/12 12:00:17+00,1800 - 1899 BLOCK OF CENTRAL PLACE NE,5,5.0,506.0,Cluster 23,Precinct 76,38.912520,-76.985590,0,...,0,0,0,0,0,0,1,1,0,0


In [24]:
data_binary_encoded.to_csv("encoded_offense_psa.csv",index=False)